In [1]:
from __future__ import annotations
from data_handler import *
from analysis import *
from dsp_premitives import *
from processors import *
from render_engine import *
from pre_master_prep import *
from streaming_normalization_simulator import *
from comparison_reporting import *
from presets_recommendations import *
from logging_versioning import *
from utils import *
from mastering_orchestrator import *

Post-Mix I/O layer loaded: AudioBuffer, load_wav, save_wav, resample_poly, slice_preview, with_suffix, auto_out_path, sha256_file, print_audio_summary.
Convenience helpers loaded: make_workspace, batch_load_wavs, env_fingerprint, Manifest, write/read_manifest, register_input, register_artifact, import_mix.
Analysis layer loaded: analyze_wav/analyze_audio_array, analysis_table, plot_spectrum, plot_short_term_loudness, plot_waveform_excerpt, LUFS approx, true-peak approx, stereo & health metrics.
DSP Primitives Layer loaded:
- Gain/level: apply_gain_db, normalize_peak, normalize_lufs, measure_peak, measure_rms
- Filters: highpass_filter, lowpass_filter, bandpass_filter, shelf_filter, peaking_eq, notch_filter, tilt_eq
- Stereo: mid_side_encode, mid_side_decode, stereo_widener
- Dynamics: compressor (soft‑knee), transient_shaper
- Fades: fade_in, fade_out
- K‑weighting/LUFS approx: k_weight, lufs_integrated_approx
Patched: render_from_cache now unpacks widen_stereo tuple correctly.
Process

## MAIN

In [4]:
# MAIN: end-to-end sanity test for I/O → Analysis → DSP Primitives → Processors
# Run this cell after loading the previous cells.
# It will:
#  - create a workspace
#  - import your mix (set MIX_SRC below)
#  - analyze the original
#  - build a preview cache
#  - render a few dial presets
#  - save outputs, register artifacts
#  - compare metrics in a table
#  - plot spectrum & loudness overlays
#
# Safe to re-run; it creates a new timestamped workspace each time.

import os, numpy as np, matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd


# ---------- 1) SET YOUR SOURCE FILE HERE ----------
MIX_SRC = "/Users/itay/Documents/muxing/ITAY - CRASHING v.6 MIX ONLY.wav"   # <-- change me
MIX_SRC = "/Users/itay/Documents/muxing/ITAY - 4 CHORDS v.21 STEFAN.wav"   # <-- change me
MIX_SRC = "/Users/itay/Documents/post_mix_data/input_mix_raw/ITAY - HOTEL POLYPHONIA STEFAN.wav"   # <-- change me



In [ ]:
# ============================================
# MAIN — Full End-to-End Pipeline (fixed report paths)
# ============================================
# Saves plots AND the HTML report inside reports/assets/ to avoid SameFileError.

import os, json, numpy as np, soundfile as sf
from dataclasses import asdict

# ---------- 0) SET YOUR SOURCE FILE HERE ----------
PROJECT  = "postmix_v1"  # MIX_SRC must be set in a previous cell

# ---------- 1) Global toggles ----------
DO_STREAMING_SIM = True
RECS_LIMIT       = 3
LM_PREVIEW_LUFS  = -14.0
REPORT_REF_NAME  = "Original"

# ---------- 2) Workspace, manifest, logger, environment ----------

workspace_root = os.path.expanduser("/Users/itay/Documents/post_mix_data/PostMixRuns")

paths = make_workspace(
    base_dir=workspace_root,
    project="postmix_v1"
)
man = Manifest(project=PROJECT, workspace=paths)
logger = RunLogger.start(workspace_root, tag=PROJECT)


env    = capture_environment()
logger.log_event("env", {"environment": env})

# bring input into workspace + register/log
if not os.path.exists(MIX_SRC):
    raise FileNotFoundError(f"Set MIX_SRC to a valid path, got: {MIX_SRC}")
mix_path = import_mix(paths, MIX_SRC, alias="mix.wav")
register_input(man, mix_path, alias="mix")
register_and_log_artifact(man, logger, mix_path, kind="input", params={"alias":"mix"}, stage="import_mix")

# ---------- 3) Load, validate, analyze original ----------
x, sr = sf.read(mix_path)
ensure_audio_valid(x, "mix")

buf = load_wav(mix_path)
print_audio_summary(buf, "Original Mix")

rep_orig = analyze_wav(mix_path)
logger.log_metrics("analysis_original", {
    "sr": rep_orig.sr, "duration_s": rep_orig.duration_s,
    "peak_dbfs": rep_orig.basic["peak_dbfs"],
    "true_peak_dbfs": rep_orig.true_peak_dbfs,
    "rms_dbfs": rep_orig.basic["rms_dbfs"],
    "lufs_int": rep_orig.lufs_integrated,
    "crest_db": rep_orig.basic["crest_db"],
    "bass_%": rep_orig.bass_energy_pct,
    "air_%": rep_orig.air_energy_pct,
    "phase_corr": rep_orig.stereo["phase_correlation"],
    "stereo_width": rep_orig.stereo["stereo_width"],
    "spectral_flatness": rep_orig.spectral_flatness,
})

# ---------- 4) Recommendations (dials) ----------
recs = recommend_from_analysis(rep_orig)
print(recommendation_summary(recs))

variants_plan = build_premaster_plan_from_recs(recs, limit=RECS_LIMIT, prefix="PM")
variants_plan.insert(0, ("PM0_Transparent", DialState(bass=0, punch=0, clarity=0, air=0, width=0)))
logger.log_params("recommendations", {
    "plan": [(name, asdict(d)) for (name, d) in variants_plan]
}, code_versions={"presets_recs": CODE_VERSIONS["presets_recs"]})

# ---------- 5) RenderEngine: preprocess + premaster variants ----------
engine = RenderEngine(x, sr, preprocess=PreprocessConfig(low_cutoff=120.0, kick_lo=40.0, kick_hi=110.0))
pre_meta = engine.preprocess()
logger.log_params("render_preprocess_cache", pre_meta, code_versions={"processors": CODE_VERSIONS["processors"], "render_engine": CODE_VERSIONS["render_engine"]})

# A) classic premaster prep (HPF 20 Hz + peak -6 dBFS)
prep_audio, prep_info = premaster_prep(x, sr, target_peak_dbfs=CFG.prep_peak_target_dbfs, hpf_hz=CFG.prep_hpf_hz)
premaster_prep_path = os.path.join(paths.outputs, "premaster", "premaster_prep.wav")
os.makedirs(os.path.dirname(premaster_prep_path), exist_ok=True)
save_wav_24bit(premaster_prep_path, prep_audio, sr)
register_and_log_artifact(man, logger, premaster_prep_path, kind="premaster", params=prep_info, stage="premaster_prep")

# B) dial-based premaster variants
variant_dir = os.path.join(paths.outputs, "premasters")
opts = RenderOptions(target_peak_dbfs=CFG.render_peak_target_dbfs, bit_depth=CFG.default_bit_depth, hpf_hz=None, save_headroom_first=False)
var_metas = engine.commit_variants(variant_dir, variants_plan, opts=opts)
for meta in var_metas:
    register_and_log_artifact(man, logger, meta["out_path"], kind="premaster", params=meta, stage=f"variant__{os.path.basename(meta['out_path'])}")

# Choose a “premaster for mastering”
premaster_choice_path = var_metas[1]["out_path"] if len(var_metas) > 1 else premaster_prep_path  # skip transparent baseline

# true-peak guard before sending to mastering
y_in, sr2 = sf.read(premaster_choice_path)
tpres = safe_true_peak(y_in, sr2, ceiling_db=CFG.tp_ceiling_db)  # TPGuardResult
premaster_for_mastering_path = os.path.join(paths.outputs, "premaster", "premaster_for_mastering.wav")
save_wav_24bit(premaster_for_mastering_path, tpres.out, sr2)
tp_meta = {"gain_db": float(tpres.gain_db), "in_dbtp": float(tpres.in_dbtp), "out_dbtp": float(tpres.out_dbtp), "ceiling_db": float(tpres.ceiling_db)}
register_and_log_artifact(man, logger, premaster_for_mastering_path, kind="premaster", params={"tp_guard": tp_meta}, stage="premaster_for_mastering")

# ---------- 6) Mastering Orchestrator (Local provider; LANDR stub available) ----------
orch = MasteringOrchestrator(paths, man)
styles = [(s, strength) for (s, strength, why) in recommend_mastering_styles_from_metrics(analyze_wav(premaster_for_mastering_path))]
providers = [LocalMasterProvider(bit_depth=CFG.default_bit_depth)]
logger.log_params("mastering_styles", {"styles": styles}, code_versions={"orchestrator": CODE_VERSIONS["orchestrator"]})

master_results = orch.run(
    premaster_path=premaster_for_mastering_path,
    providers=providers,
    styles=styles,
    out_tag="master",
    level_match_preview_lufs=LM_PREVIEW_LUFS
)
master_paths = [r.out_path for r in master_results]

# ---------- 7) Streaming Normalization Simulator (as-heard files) ----------
stream_paths = []
stream_df = None
if DO_STREAMING_SIM and master_paths:
    stream_outdir = os.path.join(paths.outputs, "stream_previews")
    stream_paths, stream_df = simulate_and_export_for_platforms(
        input_path=master_paths[0],
        out_dir=stream_outdir,
        profiles=default_streaming_profiles(),
        bit_depth=CFG.default_bit_depth,
        register_to_manifest=(man, "stream_sim")
    )
    logger.log_event("stream_sim_summary", {"rows": None if stream_df is None else len(stream_df)})

# ---------- 8) Reporting (metrics, plots, HTML) ----------
compare_files = [mix_path, premaster_for_mastering_path] + master_paths
compare_files = [p for p in compare_files if os.path.exists(p)]

cfg_cmp = CompareConfig(
    preview_seconds=CFG.preview_seconds,
    nfft=CFG.nfft,
    reference_name=REPORT_REF_NAME
)

# Save plots AND HTML into reports/assets/ (no cross-copy collisions)
reports_assets_dir = os.path.join(paths.reports, "assets")
os.makedirs(reports_assets_dir, exist_ok=True)

bundle = write_report_bundle(
    file_paths=compare_files,
    reports_dir=reports_assets_dir,               # plots + HTML here
    cfg=cfg_cmp,
    manifest=man,
    report_name="comparison_report.html",         # filename only
    extra_notes="Auto-generated end-to-end run."
)

# Optional enhanced HTML (keep it in the same assets dir)
try:
    html_enh = write_report_html_enhanced(
        bundle["summary_df"], bundle["deltas_df"], bundle["plots"],
        os.path.join(reports_assets_dir, "comparison_report_enhanced.html"),
        title="Post-Mix Comparison Report",
        extra_notes="Client review pack",
        code_versions=CODE_VERSIONS,
        dial_snapshot=variants_plan[1][1].__dict__ if len(variants_plan)>1 else {}
    )
    register_and_log_artifact(man, logger, html_enh, kind="report", params={"enhanced": True}, stage="compare_html_enhanced")
except NameError:
    pass

# ---------- 9) Reproducibility bundle ----------
zip_path = os.path.join(paths.reports, "bundles", f"{logger.run_id}.zip")
repro_zip = make_repro_zip(
    zip_path,
    workspace_root=paths.root,
    run_logger=logger,
    env_info=env,
    inputs=[mix_path],
    outputs=list(set([premaster_prep_path, premaster_for_mastering_path] + [m for m in master_paths] + stream_paths)),
    extra_jsons={"code_versions": CODE_VERSIONS, "recommendations": [(n, asdict(d)) for (n,d) in variants_plan]},
    readme_text="Bundle created by MAIN end-to-end run."
)
register_and_log_artifact(man, logger, repro_zip, kind="bundle", params={"run_id": logger.run_id}, stage="repro_zip")

# ---------- 10) Finalize ----------
logger.write_summary({
    "project": PROJECT,
    "run_id": logger.run_id,
    "mix": mix_path,
    "premaster_choice": premaster_for_mastering_path,
    "masters": master_paths,
    "stream_previews": stream_paths,
    "report": bundle["html_path"],   # lives in reports/assets/
    "env_sha": json_sha256(env),
})
write_manifest(man)

print("\n=== DONE ===")
print("Workspace:", paths.root)
print("Premaster (for mastering):", premaster_for_mastering_path)
print("Masters:", *master_paths, sep="\n - ")
if stream_paths: print("As-heard previews:", *stream_paths, sep="\n - ")
print("Report:", bundle["html_path"])
print("Repro bundle:", repro_zip)


Workspace created at: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_v1_20250820-191920
Imported mix → /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_v1_20250820-191920/inputs/mix.wav
Original Mix: sr=44100 | ch=2 | dur=322.742s | peak=0.833367 | rms=0.101539
  path: /Users/itay/Documents/post_mix_data/PostMixRuns/postmix_v1_20250820-191920/inputs/mix.wav
  sha256: 88cb9cf26c7530ec...
  src dtype: int32 | src ch: 2
- Punch Up the Transients (priority 2): B16 P38 C10 A14 W6
    • crest 18.2 dB (dynamic) and LUFS -21.7 (quiet) → add transient definition
    • small air lift to help intelligibility
